# Extract Content from Your File

This notebook demonstrate you can use Content Understanding API to extract semantic content from multimodal files.

Source: https://github.com/Azure-Samples/azure-ai-content-understanding-python.git
further expanded for training purposes

## Create Azure AI Content Understanding Client

> The AzureContentUnderstandingClient is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [1]:
import logging
import json
import os
import sys
import uuid
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [2]:
load_dotenv(override=True, dotenv_path=find_dotenv())

True

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
AZURE_AI_ENDPOINT = os.getenv("AZURE_CU_ENDPOINT_NEW")
AZURE_AI_API_VERSION = os.getenv("AZURE_CU_API_VERSION_NEW", "2025-05-01-preview")
print(f"Current Azure Content Understanding endpoint: {AZURE_AI_ENDPOINT}")
print(f"Current Azure Content Understanding API version: {AZURE_AI_API_VERSION}")

Current Azure Content Understanding endpoint: https://epaifhub6672084982.cognitiveservices.azure.com/
Current Azure Content Understanding API version: 2025-05-01-preview


In [5]:
# only if necessary, add the parent directory to the path to use shared modules
# parent_dir = Path(Path.cwd()).parent
# sys.path.append(str(parent_dir))

# import the utility class AzureContentUnderstandingClient, which is a wrapper around the Azure Content Understanding REST API client
from python.content_understanding_client_NEW import AzureContentUnderstandingClient

In [6]:
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS


In [7]:
client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/content_extraction", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.23.0 Python/3.13.4 (Windows-11-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential


In [8]:
# Utility function to save images
from PIL import Image
from io import BytesIO
import re

def save_image(image_id: str, response):
    raw_image = client.get_image_from_analyze_operation(analyze_response=response,
        image_id=image_id
    )
    image = Image.open(BytesIO(raw_image))
    # image.show()
    Path(".cache").mkdir(exist_ok=True)
    image.save(f".cache/{image_id}.jpg", "JPEG")

## Document Content

Content Understanding API is designed to extract all textual content from a specified document file. In addition to text extraction, it conducts a comprehensive layout analysis to identify and categorize tables and figures within the document. The output is then presented in a structured markdown format, ensuring clarity and ease of interpretation.



In [9]:
ANALYZER_SAMPLE_FILE = 'assets/docs/invoice-logic-apps-tutorial.pdf'
ANALYZER_ID = 'prebuilt-documentAnalyzer'

# Analyzer file
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

INFO:python.content_understanding_client_NEW:Analyzing file assets/docs/invoice-logic-apps-tutorial.pdf with analyzer: prebuilt-documentAnalyzer
INFO:python.content_understanding_client_NEW:Request 46f4dccf-769c-4d78-9a92-9aa35bc6b083 in progress ...
INFO:python.content_understanding_client_NEW:Request 46f4dccf-769c-4d78-9a92-9aa35bc6b083 in progress ...
INFO:python.content_understanding_client_NEW:Request result is ready after 4.86 seconds.


{
  "id": "46f4dccf-769c-4d78-9a92-9aa35bc6b083",
  "status": "Succeeded",
  "result": {
    "analyzerId": "prebuilt-documentAnalyzer",
    "apiVersion": "2025-05-01-preview",
    "createdAt": "2025-06-14T10:45:34Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "CONTOSO LTD.\n\n\n# INVOICE\n\nContoso Headquarters\n123 456th St\nNew York, NY, 10001\n\nINVOICE: INV-100\n\nINVOICE DATE: 11/15/2019\n\nDUE DATE: 12/15/2019\n\nCUSTOMER NAME: MICROSOFT CORPORATION\n\nSERVICE PERIOD: 10/14/2019 - 11/14/2019\n\nCUSTOMER ID: CID-12345\n\nMicrosoft Corp\n123 Other St,\nRedmond WA, 98052\n\nBILL TO:\n\nMicrosoft Finance\n\n123 Bill St,\n\nRedmond WA, 98052\n\nSHIP TO:\n\nMicrosoft Delivery\n\n123 Ship St,\n\nRedmond WA, 98052\n\nSERVICE ADDRESS:\nMicrosoft Services\n123 Service St,\nRedmond WA, 98052\n\n\n<table>\n<tr>\n<th>SALESPERSON</th>\n<th>P.O. NUMBER</th>\n<th>REQUISITIONER</th>\n<th>SHIPPED VIA</th>\n<th>F.O.B. POINT</th>\n<th>TERMS</th>\n</tr>\n<tr>\n<td></td>\n<td>PO

> The markdown output contains layout information, which is very useful for Retrieval-Augmented Generation (RAG) scenarios. You can paste the markdown into a viewer such as Visual Studio Code and preview the layout structure.

In [10]:
print(result_json["result"]["contents"][0]["markdown"])

CONTOSO LTD.


# INVOICE

Contoso Headquarters
123 456th St
New York, NY, 10001

INVOICE: INV-100

INVOICE DATE: 11/15/2019

DUE DATE: 12/15/2019

CUSTOMER NAME: MICROSOFT CORPORATION

SERVICE PERIOD: 10/14/2019 - 11/14/2019

CUSTOMER ID: CID-12345

Microsoft Corp
123 Other St,
Redmond WA, 98052

BILL TO:

Microsoft Finance

123 Bill St,

Redmond WA, 98052

SHIP TO:

Microsoft Delivery

123 Ship St,

Redmond WA, 98052

SERVICE ADDRESS:
Microsoft Services
123 Service St,
Redmond WA, 98052


<table>
<tr>
<th>SALESPERSON</th>
<th>P.O. NUMBER</th>
<th>REQUISITIONER</th>
<th>SHIPPED VIA</th>
<th>F.O.B. POINT</th>
<th>TERMS</th>
</tr>
<tr>
<td></td>
<td>PO-3333</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
</table>


<table>
<tr>
<th>DATE</th>
<th>ITEM CODE</th>
<th>DESCRIPTION</th>
<th>QTY</th>
<th>UM</th>
<th>PRICE</th>
<th>TAX</th>
<th>AMOUNT</th>
</tr>
<tr>
<td>3/4/2021</td>
<td>A123</td>
<td>Consulting Services</td>
<td>2</td>
<td>hours</td>
<td>$30.00</td>
<td>10%</td>
<td>$60.00<

In [11]:
# save the markdown as a file
markdown_file = Path("result.md")
if not markdown_file.exists():
    with open(markdown_file, "w") as f:
        f.write(result_json["result"]["contents"][0]["markdown"])

> You can get the layout information, including ```words/lines``` in the pagesnode and paragraphs info in ```paragraphs```, and ```tables``` in the table.

In [12]:
print(json.dumps(result_json["result"]["contents"][0], indent=2))


{
  "markdown": "CONTOSO LTD.\n\n\n# INVOICE\n\nContoso Headquarters\n123 456th St\nNew York, NY, 10001\n\nINVOICE: INV-100\n\nINVOICE DATE: 11/15/2019\n\nDUE DATE: 12/15/2019\n\nCUSTOMER NAME: MICROSOFT CORPORATION\n\nSERVICE PERIOD: 10/14/2019 - 11/14/2019\n\nCUSTOMER ID: CID-12345\n\nMicrosoft Corp\n123 Other St,\nRedmond WA, 98052\n\nBILL TO:\n\nMicrosoft Finance\n\n123 Bill St,\n\nRedmond WA, 98052\n\nSHIP TO:\n\nMicrosoft Delivery\n\n123 Ship St,\n\nRedmond WA, 98052\n\nSERVICE ADDRESS:\nMicrosoft Services\n123 Service St,\nRedmond WA, 98052\n\n\n<table>\n<tr>\n<th>SALESPERSON</th>\n<th>P.O. NUMBER</th>\n<th>REQUISITIONER</th>\n<th>SHIPPED VIA</th>\n<th>F.O.B. POINT</th>\n<th>TERMS</th>\n</tr>\n<tr>\n<td></td>\n<td>PO-3333</td>\n<td></td>\n<td></td>\n<td></td>\n<td></td>\n</tr>\n</table>\n\n\n<table>\n<tr>\n<th>DATE</th>\n<th>ITEM CODE</th>\n<th>DESCRIPTION</th>\n<th>QTY</th>\n<th>UM</th>\n<th>PRICE</th>\n<th>TAX</th>\n<th>AMOUNT</th>\n</tr>\n<tr>\n<td>3/4/2021</td>\n<td>A123</td

> This statement allows you to get structural information of the tables in the documents.

In [14]:
nr_tables = len(result_json["result"]["contents"][0]["tables"])
print(f"Number of tables found: {nr_tables}")

Number of tables found: 3


In [31]:
print(json.dumps(result_json["result"]["contents"][0]["tables"], indent=2))

[
  {
    "rowCount": 2,
    "columnCount": 6,
    "cells": [
      {
        "kind": "columnHeader",
        "rowIndex": 0,
        "columnIndex": 0,
        "rowSpan": 1,
        "columnSpan": 1,
        "content": "SALESPERSON",
        "source": "D(1,0.5389,4.5514,1.7505,4.5514,1.7505,4.8364,0.5389,4.8364)",
        "span": {
          "offset": 512,
          "length": 11
        },
        "elements": [
          "/paragraphs/19"
        ]
      },
      {
        "kind": "columnHeader",
        "rowIndex": 0,
        "columnIndex": 1,
        "rowSpan": 1,
        "columnSpan": 1,
        "content": "P.O. NUMBER",
        "source": "D(1,1.7505,4.5514,3.2399,4.5591,3.2399,4.8364,1.7505,4.8364)",
        "span": {
          "offset": 533,
          "length": 11
        },
        "elements": [
          "/paragraphs/20"
        ]
      },
      {
        "kind": "columnHeader",
        "rowIndex": 0,
        "columnIndex": 2,
        "rowSpan": 1,
        "columnSpan": 1,
        

## Audio Content
Our API output facilitates detailed analysis of spoken language, allowing developers to utilize the data for various applications, such as voice recognition, customer service analytics, and conversational AI. The structure of the output makes it easy to extract and analyze different components of the conversation for further processing or insights.

1. Speaker Identification: Each phrase is attributed to a specific speaker (in this case, "Speaker 2"). This allows for clarity in conversations with multiple participants.
1. Timing Information: Each transcription includes precise timing data:
    - startTimeMs: The time (in milliseconds) when the phrase begins.
    - endTimeMs: The time (in milliseconds) when the phrase ends.
    This information is crucial for applications like video subtitles, allowing synchronization between the audio and the text.
1. Text Content: The actual spoken text is provided, which in this instance is "Thank you for calling Woodgrove Travel." This is the main content of the transcription.
1. Confidence Score: Each transcription phrase includes a confidence score (0.933 in this case), indicating the likelihood that the transcription is accurate. A higher score suggests greater reliability.
1. Word-Level Breakdown: The transcription is further broken down into individual words, each with its own timing information. This allows for detailed analysis of speech patterns and can be useful for applications such as language processing or speech recognition improvement.
1. Locale Specification: The locale is specified as "en-US," indicating that the transcription is in American English. This is important for ensuring that the transcription algorithms account for regional dialects and pronunciations.


In [33]:
ANALYZER_SAMPLE_FILE = 'assets/audio/audio.wav'
ANALYZER_ID = 'prebuilt-audioAnalyzer'

# Analyzer file
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

INFO:python.content_understanding_client_NEW:Analyzing file assets/audio/audio.wav with analyzer: prebuilt-audioAnalyzer
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understanding_client_NEW:Request 49848917-9f8e-4142-8664-48a77710f4c6 in progress ...
INFO:python.content_understandi

{
  "id": "49848917-9f8e-4142-8664-48a77710f4c6",
  "status": "Succeeded",
  "result": {
    "analyzerId": "prebuilt-audioAnalyzer",
    "apiVersion": "2025-05-01-preview",
    "createdAt": "2025-06-14T11:04:38Z",
    "stringEncoding": "utf8",
    "warnings": [],
    "contents": [
      {
        "markdown": "# Audio: 00:00.000 => 01:54.670\n\nTranscript\n```\nWEBVTT\n\n00:00.080 --> 00:02.160\n<v Speaker 1>Thank you for calling Woodgrove Travel.\n\n00:02.960 --> 00:04.560\n<v Speaker 1>My name is Isabella Taylor.\n\n00:05.280 --> 00:06.880\n<v Speaker 1>How may I assist you today?\n\n00:07.680 --> 00:10.240\n<v Speaker 2>Hi Isabella, my name is John Smith.\n\n00:11.040 --> 00:17.920\n<v Speaker 2>I recently traveled from New York City to Los Angeles on a business trip, and I had a terrible experience with my flight.\n\n00:18.720 --> 00:20.880\n<v Speaker 1>I'm sorry to hear that, John.\n\n00:21.680 --> 00:27.200\n<v Speaker 1>Could you please provide me with the details of your flight

## Video Content
Video output provides detailed information about audiovisual content, specifically video shots. Here are the key features it offers:

1. Shot Information: Each shot is defined by a start and end time, along with a unique identifier. For example, Shot 0:0.0 to 0:2.800 includes a transcript and key frames.
1. Transcript: The API includes a transcript of the audio, formatted in WEBVTT, which allows for easy synchronization with the video. It captures spoken content and specifies the timing of the dialogue.
1. Key Frames: It provides a series of key frames (images) that represent important moments in the video shot, allowing users to visualize the content at specific timestamps.
1. Description: Each shot is accompanied by a description, providing context about the visuals presented. This helps in understanding the scene or subject matter without watching the video.
1. Audio Visual Metadata: Details about the video such as dimensions (width and height), type (audiovisual), and the presence of key frame timestamps are included.
1. Transcript Phrases: The output includes specific phrases from the transcript, along with timing and speaker information, enhancing the usability for applications like closed captioning or search functionalities.

In [34]:
ANALYZER_SAMPLE_FILE = 'assets/video/FlightSimulator.mp4'
ANALYZER_ID = 'prebuilt-videoAnalyzer'

# Analyzer file
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

# Save keyframes (optional)
keyframe_ids = set()
result_data = result_json.get("result", {})
contents = result_data.get("contents", [])

# Iterate over contents to find keyframes if available
for content in contents:
    # Extract keyframe IDs from "markdown" if it exists and is a string
    markdown_content = content.get("markdown", "")
    if isinstance(markdown_content, str):
        keyframe_ids.update(re.findall(r"(keyFrame\.\d+)\.jpg", markdown_content))

# Output the results
print("Unique Keyframe IDs:", keyframe_ids)

# Save all keyframe images
for keyframe_id in keyframe_ids:
    save_image(keyframe_id, response)

INFO:python.content_understanding_client_NEW:Analyzing file assets/video/FlightSimulator.mp4 with analyzer: prebuilt-videoAnalyzer
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_understanding_client_NEW:Request 835d2b67-0b9d-4fff-97ab-83a5c75c426a in progress ...
INFO:python.content_u

{
  "id": "835d2b67-0b9d-4fff-97ab-83a5c75c426a",
  "status": "Succeeded",
  "result": {
    "analyzerId": "prebuilt-videoAnalyzer",
    "apiVersion": "2025-05-01-preview",
    "createdAt": "2025-06-14T11:06:15Z",
    "stringEncoding": "utf8",
    "warnings": [],
    "contents": [
      {
        "markdown": "# Video: 00:00.000 => 00:43.866\nWidth: 1080\nHeight: 608\n\n## Segment 1: 00:00.000 => 00:07.367\nThe video begins with a scenic aerial view of an island, showcasing the collaboration between Flight Simulator and Microsoft Azure AI. This is followed by a visual of sound waves, representing the neural TTS technology discussed in the audio transcript. The audio explains the importance of good data for neural TTS and mentions the creation of a universal TTS model based on 3,000 hours of data.\n\nTranscript\n```\nWEBVTT\n\n00:01.360 --> 00:06.640\n<Speaker 1>When it comes to the neural TTS, in order to get a good voice, it's better to have good data.\n\n00:07.120 --> 00:13.320\n<Spea

## Video Content with Face
This is a gated feature, please go through process [Azure AI Resource Face Gating](https://learn.microsoft.com/en-us/legal/cognitive-services/computer-vision/limited-access-identity?context=%2Fazure%2Fai-services%2Fcomputer-vision%2Fcontext%2Fcontext#registration-process) Select `[Video Indexer] Facial Identification (1:N or 1:1 matching) to search for a face in a media or entertainment video archive to find a face within a video and generate metadata for media or entertainment use cases only` in the registration form.

In [ ]:
ANALYZER_SAMPLE_FILE = '../data/FlightSimulator.mp4'
ANALYZER_ID = 'prebuilt-videoAnalyzer'

# Analyzer file
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

### Get and Save Key Frames and Face Thumbnails

In [ ]:
# Initialize sets for unique face IDs and keyframe IDs
face_ids = set()
keyframe_ids = set()

# Extract unique face IDs safely
result_data = result_json.get("result", {})
contents = result_data.get("contents", [])

# Iterate over contents to find faces and keyframes if available
for content in contents:
    # Safely retrieve face IDs if "faces" exists and is a list
    faces = content.get("faces", [])
    if isinstance(faces, list):
        for face in faces:
            face_id = face.get("faceId")
            if face_id:
                face_ids.add(f"face.{face_id}")

    # Extract keyframe IDs from "markdown" if it exists and is a string
    markdown_content = content.get("markdown", "")
    if isinstance(markdown_content, str):
        keyframe_ids.update(re.findall(r"(keyFrame\.\d+)\.jpg", markdown_content))

# Output the results
print("Unique Face IDs:", face_ids)
print("Unique Keyframe IDs:", keyframe_ids)

# Save all face images
for face_id in face_ids:
    save_image(face_id, response)

# Save all keyframe images
for keyframe_id in keyframe_ids:
    save_image(keyframe_id, response)